
### Core Mantras for Runtime Validation & Robust Python

| **Mantra**                                                        | **Meaning (why it matters)**                                                                                                                                  |
| ----------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Validate at boundaries, not everywhere.**                       | Validate only at system edges (files, configs, user input, API responses). Inside the code: assume data is already good. Prevents over-defensive, noisy code. |
| **Catch only what you can fix.**                                  | If you can’t add context or recover meaningfully, don’t catch the exception. Prevents masking real bugs.                                                      |
| **Let `open()` tell you the truth.**                              | Don’t pre-check `exists()`/`isfile()`. Avoid race conditions and double work; trust the OS error.                                                             |
| **Duck typing means validating capability, not type.**            | Check “does it have `.read()`?” instead of checking for `TextIOWrapper`. Supports numpy, pandas, Path, StringIO, etc.                                         |
| **Be strict at the edges, liberal inside.**                       | Boundary layers reject bad input; internal code stays lean and flexible. Encourages clean architecture.                                                       |
| **Use type hints to express intention, not enforce constraints.** | Don’t turn Python into Java. Type hints are communication, not runtime safety.                                                                                |
| **Coercion is a contract, not a convenience.**                    | Only coerce data if you explicitly *want* coercion. Don’t surprise the caller.                                                                                |
| **Don’t validate states your own code can’t produce.**            | Defensive checks for impossible conditions cause more bugs and clutter.                                                                                       |
| **Raise errors early, and with caller-focused messages.**         | Good error messages point to *what the caller did wrong*, not where the code blew up.                                                                         |
| **Reject half-validated data structures.**                        | A partially validated object is more dangerous than an unvalidated one. Validate fully or not at all.                                                         |
| **Accept the minimal interface required.**                        | If you only need iteration, accept any iterable; don’t require `list`. Enables real duck typing.                                                              |
| **Don’t over-annotate.**                                          | Type signatures should stay readable, minimal, and intention-oriented. Overly rigid annotations prevent valid inputs.                                         |
| **Fail fast, fail clean.**                                        | A clean crash is better than a corrupt state hidden under "error handling."                                                                                   |
| **Don’t catch `except:` and rarely catch `Exception`.**           | Avoid swallowing `KeyboardInterrupt`, `SystemExit`, and debugging signals. Be precise.                                                                        |
| **Use plain Python types after validation.**                      | Don’t let `pydantic` or `pandera` bleed into core logic. Convert to domain objects and operate cleanly.                                                       |
| **Don’t validate just to please tests.**                          | Tests should reflect reality. Avoid imaginary constraints stemming from bad test design.                                                                      |
| **Check semantics, not just types.**                              | `n > 0` matters more than `type(n) is int`. Real bugs come from wrong values, not wrong types.                                                                |





# **1. Validate at boundaries, not everywhere.**

### ✔ Correct

```python
def load_config(path):
    # Boundary: validate here
    with open(path) as f:
        return yaml.safe_load(f)
```

### ❌ Wrong (internal over-defensiveness)

```python
def compute_statistics(data):
    if not isinstance(data, list):      # pointless internally
        raise TypeError
    ...
```

# **2. Catch only what you can fix.**

### ✔ Correct

```python
try:
    with open(path) as f:
        return f.read()
except FileNotFoundError:
    raise FileNotFoundError(f"No such file: {path}")
```

### ❌ Wrong

```python
try:
    risky()
except Exception:
    print("Something went wrong")  # hides real bug
```

# **3. Let `open()` tell you the truth.**

### ✔ Correct

```python
with open(path) as f:
    data = f.read()
```

### ❌ Wrong

```python
if os.path.exists(path):        # race condition + redundant
    with open(path) as f:
        data = f.read()
```



# **4. Duck typing means validating capability, not type.**

### ✔ Correct

```python
def load(obj):
    if hasattr(obj, "read"):
        return obj.read()
    with open(obj) as f:
        return f.read()
```

### ❌ Wrong

```python
def load(obj: io.TextIOWrapper):  # rejects valid file-like objects
    return obj.read()
```





# **5. Be strict at the edges, liberal inside.**

### ✔ Strict at boundary

```python
def parse_args(args):
    if args.n_samples <= 0:
        raise ValueError("n_samples must be positive")
```

### ✔ Liberal inside

```python
def simulate(n_samples):
    # assume validated
    return np.random.randn(n_samples)
```





# **6. Use type hints to express intention, not enforce constraints.**

### ✔ Good

```python
def center(xs: Sequence[float]) -> float:
    return (xs[0] + xs[1]) / 2
```

### ❌ Bad

```python
def center(xs: list[float]):  # needlessly rigid
    ...
```




# **7. Coercion is a contract, not a convenience.**

### ✔ Honest coercion

```python
def parse_int(x: str) -> int:
    return int(x)  # caller knows they must pass a stringified int
```

### ❌ Hidden coercion

```python
def threshold(t):
    t = float(t)    # silently accepts nonsense like t="hello"
    return t
```





# **8. Don’t validate states your own code can’t produce.**

### ✔ Good

```python
def _normalize(xs):
    # internal helper, xs is guaranteed list of floats
    mean = sum(xs) / len(xs)
    return [x - mean for x in xs]
```

### ❌ Bad

```python
def _normalize(xs):
    if xs is None:  # impossible if your callers are correct
        raise ValueError
```





# **9. Raise errors early, with caller-focused messages.**

### ✔ Good

```python
def compute_ratio(a, b):
    if b == 0:
        raise ValueError("b must be non-zero (received 0)")
    return a / b
```

### ❌ Bad

```python
def compute_ratio(a, b):
    return a / b            # ZeroDivisionError deep inside
```





# **10. Reject half-validated data structures.**

### ✔ Good

```python
def parse_point(d):
    try:
        x = float(d["x"])
        y = float(d["y"])
    except KeyError as e:
        raise ValueError(f"Missing key: {e.args[0]}")
    return x, y
```

### ❌ Bad

```python
def parse_point(d):
    x = float(d["x"])
    y = d["y"]              # unvalidated garbage slips through
    return x, y
```





# **11. Accept the minimal interface required.**

### ✔ Capability-based

```python
def first_two(xs: Sequence):
    return xs[0], xs[1]
```

### ❌ Overly strict

```python
def first_two(xs: list):    # excludes numpy arrays, tuples, etc.
    return xs[0], xs[1]
```





# **12. Don’t over-annotate.**

### ✔ Good

```python
def summarize(points: list[tuple[float, float]]):
    ...
```

### ❌ Bad

```python
def summarize(
    points: Sequence[tuple[float, float] | list[float] | np.ndarray]
) -> Mapping[str, float]:
    ...   # unreadable & unnecessary
```





# **13. Fail fast, fail clean.**

### ✔ Good

```python
def invert(x):
    if x == 0:
        raise ValueError("x cannot be zero")
    return 1 / x
```

### ❌ Bad

```python
def invert(x):
    try:
        return 1 / x
    except:
        return None     # wrong, silently hides meaning
```





# **14. Don’t catch `except:` and rarely catch `Exception`.**

### ✔ Good

```python
try:
    write_result(data)
except PermissionError:
    raise PermissionError("Cannot write to output directory")
```

### ❌ Bad

```python
try:
    write_result(data)
except Exception:
    pass  # catastrophic
```





# **15. Use plain Python types after validation.**

### ✔ Good

```python
@dataclass
class Config:
    output_dir: str
    threshold: float

config = Config(**pydantic_model.model_dump())
```

### ❌ Bad

```python
def core_algorithm(cfg: PydanticModel):  # domain logic tied to framework
    ...
```





# **16. Don’t validate just to please tests.**

### ✔ Good

```python
def compute_total(values):
    return sum(values)
```

### ❌ Bad test-driven cargo cult

```python
def compute_total(values):
    if not isinstance(values, list):   # added only because a test passed a tuple
        raise TypeError
    return sum(values)
```





# **17. Check semantics, not just types.**

### ✔ Good

```python
def set_probability(p):
    if not 0 <= p <= 1:
        raise ValueError("p must be between 0 and 1")
```

### ❌ Bad

```python
def set_probability(p: float):
    return p     # accepts -3.7 or 99.2 with no issue
```

